# Table of Content

1. Introduction of Tool
2. Features of Tool
3. Requirements of Tool
4. How to Work
5. Folders Description
6. Toy examples

### 1. Introduction of Tool

Dynamic Analyzer tool has leverage the capabilities of LibClang to analyze the dynamic 
aspects of C language programs. LibClang is considered as a stable high level C interface to clang and used for high level abstractions by traversing an AST through a cursor. 


### 2. Features of Tool

Besides dynamic aspects of a program, the tool would help to obtain the following features as follow.

	•	To extract the name, mangled name and qualified name of functions
	•	To extract the function definitions with its line number of source code.
	•	To extract the type of a function such as Template function, a method or normal function
	•	To extract the calling points location of a function along with parameters
	•	To extract the cyclomatic complexity of each function
	•	To extract the longest functions call complexity
	•	To accumulate the total cyclomatic complexity

### 3. Requirments of Tool

	•	llvm (Preferable llvm-11.0.0)
	•	Clang
	•	Python ( Supports both Python 2 ad 3)
	•	Conda 
	•	Ubuntu/macOS

### 3. How to work

Firstly open the Terminal and clone the repository as follow.

>    git clone https://github.com/HPCL/code-analysis.git

Code-analysis folder will be created on system current root path. It will be open by applying the following command.

>    cd code-analysis

In code-analysis folder execute the environments.yml file as follow.

>    conda env create –f environments.yml

The content of environments.yml is covering the dependencies which are used to execute the Dynamic analyzer tool smoothly. If you are executing environments.yml first time and initially dependency requirements are not available on your system then it will take few seconds. 

To check the status of all tests, apply the following commands

>    cd  /Users/…/code-analysis/Dynamic/tests
>    pytest

To execute a specific test, apply the following command and observe the output.

>    pytest –q –s –name example3.c test_Function_List.py

This command is passing a file example3.c as an argument with reference –name to test_Function_List.py. In case of passing more than one arguments with references, you can update conftest.py file in tests folder.

### 4. Folder Description

The hierarchy of clone folder is as follows

Code-analysis
 ------Dynamic
          ---------- examples
    ---------- src  
    ---------- tests
          ---------- README.md
          ---------- requirments.txt
  ------Static
  ------__init__.py
  -------environments.yml


src Folder:

        This folder contains the python scripts which are used to cover the different aspects of dynamic analysis.

tests Folder:

        This folder contain three types of files as follows
        1. Tests script written in python and can be executed through pytest.
        2. Toy examples of *.c to understand the functionality of scripts of src folder and are passed as an argument when a test is checked through pytest.
	    3. Conftest.py file is used to declare the arguments reference which are passed through pytest. Initially, --name reference is given for passing *.c file as argument of pytest command.


### 5. Toy Examples

### Example-1: (Example2 in tests folder)

void Large(int a, int b)
{
  if(a>=b)
     printf("large number is %d", a);
  else
     printf("Large number is %d", b);
}

void main() {
   int A=2, B=3, Larg;
   if(A == B)
      printf("Both number are same");
   else
      Large(A, B);
      Large(5,4);
   printf("Bye");
}

### Script-1 (Function_List.py) : Retreive information about all functions of example2.c

In [71]:


import csv, os, glob
import sys
import clang.cindex
from clang.cindex import Config
os.environ['DYLD_LIBRARY_PATH']= '/usr/local/Cellar/llvm/11.0.0/lib/'
#Config.set_library_path('/usr/local/Cellar/llvm/11.0.0/lib')
#Config.set_library_path('/usr/local/anaconda3/lib/python3.8/site-packages/')

def Extract_Function_Names(file_name, tu):
    List_of_Functions=[]
    col=[]
    col.append("File Name")
    col.append("Function Name")
    col.append("Mangled Name")
    col.append("Function Type")
    List_of_Functions.append(col)
    filename = tu.cursor.spelling
    for c in tu.cursor.walk_preorder():
       if c.location.file is None:
            pass
       elif c.location.file.name != filename:
            pass
       elif c.kind == clang.cindex.CursorKind.FUNCTION_DECL or c.kind==clang.cindex.CursorKind.CXX_METHOD or c.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE:
            col=[]
            col.append(file_name)
            col.append(c.spelling)
            col.append(c.mangled_name)
            if c.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE:
               col.append('Template Function')
            elif c.kind==clang.cindex.CursorKind.CXX_METHOD:
               col.append("Class Method")
            elif c.kind==clang.cindex.CursorKind.FUNCTION_DECL:
               col.append("Function")
            List_of_Functions.append(col)
    return List_of_Functions

idx = clang.cindex.Index.create()
os.chdir("/Users/shussain/code-analysis/dynamic/tests")
tu = idx.parse("example2.c", args='-xc++ --std=c++11'.split())
List_of_Functions = Extract_Function_Names("example2.c", tu)
for f in List_of_Functions:
    print (f)


['File Name', 'Function Name', 'Mangled Name', 'Function Type']
['example2.c', 'Large', '__Z5Largeii', 'Function']
['example2.c', 'main', '_main', 'Function']


### Script-2 (Function_Definition_Location.py) : Retreive information functions definitions and their location of example2.c

In [63]:
import csv, os, glob
import sys
import clang.cindex
from clang.cindex import Config
os.environ['DYLD_LIBRARY_PATH']= '/usr/local/Cellar/llvm/11.0.0/lib/'
File_Content_Array=[]
def File_to_Array(file_name):
    with open(file_name) as file:
        for line in file:
           File_Content_Array.append(line)
    #return File_Content_Array

def Extract_Line_Column(cursor):
    x=str(cursor.location)
    y=x.split(',')
    line = y[1]
    column=y[2]
    line = line.split()
    line=line[1]
    column=column.split()
    column = column[1]
    return line, column

def Extract_Function_Qualified_Name(cursor):
     if cursor is None:
        return ''
     elif cursor.kind== clang.cindex.CursorKind.TRANSLATION_UNIT:
        return ''
     else:
         res = Extract_Function_Qualified_Name(cursor.semantic_parent)
         if res != '':
            return res + '::' + cursor.spelling
     return cursor.spelling
def Extract_Function_Definition_Location(file_name, tu):
    List_of_Functions=[]
    File_to_Array(file_name)
    # Heading row for the list of functions
    col=[]
    col.append("File Name")
    col.append("Function Name")
    col.append("Function Qualified Name")
    col.append("Function Definition")
    col.append("Line Number")
    col.append("Column Number")
    List_of_Functions.append(col)

    filename = tu.cursor.spelling
    for c in tu.cursor.walk_preorder():
       if c.location.file is None:
            pass
       elif c.location.file.name != filename:
            pass
       elif c.kind == clang.cindex.CursorKind.FUNCTION_DECL or c.kind==clang.cindex.CursorKind.CXX_METHOD or c.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE:
            col=[]
            col.append(file_name)
            col.append(c.spelling)
            col.append(Extract_Function_Qualified_Name(c))
            lin1, col1 = Extract_Line_Column(c)
            col1=col1[:-1]
            st=File_Content_Array[int(lin1)-1]
            col.append(st.strip())
            col.append(lin1)
            col.append(col1)
            List_of_Functions.append(col)

    return List_of_Functions
idx = clang.cindex.Index.create()
os.chdir("/Users/shussain/code-analysis/dynamic/tests")
tu = idx.parse("example2.c", args='-xc++ --std=c++11'.split())
List_of_Function = Extract_Function_Definition_Location("example2.c", tu)
for f in List_of_Function:
    print (f)


['File Name', 'Function Name', 'Function Qualified Name', 'Function Definition', 'Line Number', 'Column Number']
['example2.c', 'Large', 'Large', 'void Large(int a, int b)', '1', '6']
['example2.c', 'main', 'main', 'void main() {', '9', '6']


### Script-3 (Function_Definition_Calls.py) : Retreive the calling points of functions including calling point definitions of example2.c

In [72]:
import csv, os, glob
import sys
import clang.cindex
from clang.cindex import Config
os.environ['DYLD_LIBRARY_PATH']= '/usr/local/Cellar/llvm/11.0.0/lib/'
File_Content_Array=[]
Function_Complexity=[]
List_of_Functions=[]
Function_List=[]
def File_to_Array(file_name):
    with open(file_name) as file:
        for line in file:
           File_Content_Array.append(line)
    #return File_Content_Array
def Extract_Line_Column(cursor):
    x=str(cursor.location)
    y=x.split(',')
    line = y[1]
    column=y[2]
    line = line.split()
    line=line[1]
    column=column.split()
    column = column[1]
    return line, column
def Extract_Function_List(tu):
    filename = tu.cursor.spelling
    for c in tu.cursor.walk_preorder():
       if c.location.file is None:
            pass
       elif c.location.file.name != filename:
            pass
       elif c.kind == clang.cindex.CursorKind.FUNCTION_DECL or c.kind==clang.cindex.CursorKind.CXX_METHOD or c.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE:
            Function_List.append(c.spelling)
def Extract_Function_Definition_Calls(file_name, tu):
    Extract_Function_List(tu)
    Final_List=[]
    col=[]
    col.append("Function Name")
    col.append("Name Space")
    col.append("Calling Point")
    col.append("Line#")
    col.append("Column#")
    Final_List.append(col)
    Name_Space="Anonymous NameSpace"
    File_to_Array(file_name)
    filename = tu.cursor.spelling
    for c in tu.cursor.get_tokens():
       if c.location.file is None:
            pass
       elif c.location.file.name != filename:
            pass
       elif c.cursor.kind == clang.cindex.CursorKind.NAMESPACE:
            Name_Space=c.spelling
       elif (c.cursor.kind == clang.cindex.CursorKind.DECL_REF_EXPR or  c.cursor.kind == clang.cindex.CursorKind.CALL_EXPR or  c.cursor.kind == clang.cindex.CursorKind.OVERLOADED_DECL_REF) and c.spelling in Function_List: # or  c.kind==clang.cindex.TokenKind.IDENTIFIER) and c.spelling in Function_List:
            lin1, col1 = Extract_Line_Column(c)
            st1=File_Content_Array[int(lin1)-1]
            col=[]
            col.append(c.spelling)
            col.append(Name_Space)
            col.append(st1)
            col.append(lin1)
            col.append(col1)
            if col not in Final_List:
                Final_List.append(col)
            else:
                col=[]
    return Final_List
idx = clang.cindex.Index.create()
os.chdir("/Users/shussain/code-analysis/dynamic/tests")
tu = idx.parse("example2.c", args='-xc++ --std=c++11'.split())
List_of_Function_Calls = Extract_Function_Definition_Calls("example2.c", tu)
for f in List_of_Function_Calls:
    print (f)


['Function Name', 'Name Space', 'Calling Point', 'Line#', 'Column#']
['Large', 'Anonymous NameSpace', '   Large(A, B);\n', '14', '4>']
['Large', 'Anonymous NameSpace', '   Large(5,4);\n', '15', '4>']


### Script-4 (Function_Complexity.py) : Retreive the information about functions and their complexities of example2.c

In [73]:
import csv, os, glob
import sys
import clang.cindex
from clang.cindex import Config
os.environ['DYLD_LIBRARY_PATH']= '/usr/local/Cellar/llvm/11.0.0/lib/'
File_Content_Array=[]
Function_Complexity=[]
List_of_Functions=[]
Function_List=[]
Cursor_Types = {clang.cindex.CursorKind.IF_STMT, clang.cindex.CursorKind.WHILE_STMT, clang.cindex.CursorKind.FOR_STMT,clang.cindex.CursorKind.DEFAULT_STMT,clang.cindex.CursorKind.CASE_STMT, clang.cindex.CursorKind.COMPOUND_STMT}
Keywords= {"if", "case",  "default", "for", "while", "else"}
# A function to copy the content of file into an array
def File_to_Array(file_name):
    with open(file_name) as file:
        for line in file:
           File_Content_Array.append(line)
    #return File_Content_Array
def Extract_Function_Complexity(tu):
     cnt=0
     flag=False
     filename = tu.cursor.spelling
     fn=""
     for c in tu.cursor.get_tokens():
         if c.location.file is None:
              pass
         elif c.location.file.name != filename:
              pass
         else:
              cond1=c.cursor.kind in Cursor_Types
              cond2=c.spelling in Keywords
              if (c.cursor.kind==clang.cindex.CursorKind.FUNCTION_DECL or c.cursor.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE) and c.spelling in Function_List:
                     flag=True
                     fn=c.spelling
                     cnt=cnt+1
              if cond1==True and cond2==True and (cnt<=len(Function_Complexity) and Function_List[cnt-1] ==fn):# and len(Function_List)==len(Function_Complexity)):
                     temp=Function_Complexity[cnt-1]
                     temp=temp+1
                     Function_Complexity[cnt-1]=temp
def Extract_Line_Column(cursor):
    x=str(cursor.location)
    y=x.split(',')
    line = y[1]
    column=y[2]
    line = line.split()
    line=line[1]
    column=column.split()
    column = column[1]
    return line, column
def Extract_Function_Qualified_Name(cursor):
     if cursor is None:
        return ''
     elif cursor.kind== clang.cindex.CursorKind.TRANSLATION_UNIT:
        return ''
     else:
         res = Extract_Function_Qualified_Name(cursor.semantic_parent)
         if res != '':
            return res + '::' + cursor.spelling
     return cursor.spelling

def Merge_Function_Complexity(Functions, Complexity):
    Final_List=[]
    # Heading row for the list of functions
    col=[]
    col.append("File Name")
    col.append("Function Name")
    col.append("Function Qualified Name")
    col.append("Mangled Name")
    col.append("Name Space")
    col.append("Function Definition")
    col.append("Line Number")
    col.append("Column Number")
    col.append("Cyclomatic Complexity")
    Final_List.append(col)
    cnt=0
    for func in Functions:
        col=[]
        col.append(func[0])
        col.append(func[1])
        col.append(func[2])
        col.append(func[3])
        col.append(func[4])
        col.append(func[5])
        col.append(func[6])
        col.append(func[7])
        col.append(Complexity[cnt])
        Final_List.append(col)
        cnt=cnt+1
    return Final_List
def Extract_Function_Definition_Location(file_name, tu):
    Name_Space="Anonymous NameSpace"
    File_to_Array(file_name)
    filename = tu.cursor.spelling
    for c in tu.cursor.walk_preorder():
       if c.location.file is None:
            pass
       elif c.location.file.name != filename:
            pass
       elif c.kind == clang.cindex.CursorKind.NAMESPACE:
            Name_Space=c.spelling
       elif c.kind == clang.cindex.CursorKind.FUNCTION_DECL or c.kind==clang.cindex.CursorKind.CXX_METHOD or c.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE:
            col=[]
            col.append(file_name)
            col.append(c.spelling)
            col.append(Extract_Function_Qualified_Name(c))
            col.append(c.mangled_name)
            col.append(Name_Space)
            lin1, col1 = Extract_Line_Column(c)
            col1=col1[:-1]
            st=File_Content_Array[int(lin1)-1]
            col.append(st.strip())
            col.append(lin1)
            col.append(col1)
            List_of_Functions.append(col)
            Function_List.append(c.spelling)
            Function_Complexity.append(0)
    Extract_Function_Complexity(tu)
    Final_List = Merge_Function_Complexity(List_of_Functions, Function_Complexity)
    return Final_List
idx = clang.cindex.Index.create()
os.chdir("/Users/shussain/code-analysis/dynamic/tests")
tu = idx.parse("example2.c", args='-xc++ --std=c++11'.split())
List_of_Function_Complexity = Extract_Function_Definition_Location("example2.c", tu)
for f in List_of_Function_Complexity:
    print (f)


['File Name', 'Function Name', 'Function Qualified Name', 'Mangled Name', 'Name Space', 'Function Definition', 'Line Number', 'Column Number', 'Cyclomatic Complexity']
['example2.c', 'Large', 'Large', '__Z5Largeii', 'Anonymous NameSpace', 'void Large(int a, int b)', '1', '6', 2]
['example2.c', 'main', 'main', '_main', 'Anonymous NameSpace', 'void main() {', '9', '6', 2]


### Script-5 (Called_Calling_Function.py) : Retreive the information about called and calling points of all functions of example2.c

In [ ]:
import csv, os, glob
import sys
import clang.cindex
from clang.cindex import Config
os.environ['DYLD_LIBRARY_PATH']= '/usr/local/Cellar/llvm/11.0.0/lib/'
File_Content_Array=[]
Function_Complexity=[]
List_of_Functions=[]
Function_List=[]
    with open(file_name) as file:
        for line in file:
           File_Content_Array.append(line)
    #return File_Content_Array


def Extract_Line_Column(cursor):
    x=str(cursor.location)
    y=x.split(',')
    line = y[1]
    column=y[2]
    line = line.split()
    line=line[1]
    column=column.split()
    column = column[1]
    return line, column

def Extract_Function_Qualified_Name(cursor):
     if cursor is None:
        return ''
     elif cursor.kind== clang.cindex.CursorKind.TRANSLATION_UNIT:
        return ''
     else:
         res = Extract_Function_Qualified_Name(cursor.semantic_parent)
         if res != '':
            return res + '::' + cursor.spelling
     return cursor.spelling

def Extract_Function_List(tu):
    filename = tu.cursor.spelling
    for c in tu.cursor.walk_preorder():
       if c.location.file is None:
            pass
       elif c.location.file.name != filename:
            pass
       elif c.kind == clang.cindex.CursorKind.FUNCTION_DECL or c.kind==clang.cindex.CursorKind.CXX_METHOD or c.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE:
            Function_List.append(c.spelling)
def Extract_Function_Definition_Calls(file_name, tu):
    Extract_Function_List(tu)
    Final_List=[]
    col=[]
    col.append("Calling Function Name")
    col.append("Function Definition")
    col.append("Line#")
    col.append("Column#")
    col.append("Name Space")
    col.append("Called Function")
    col.append("Called Function Definition")
    col.append("Line#")
    col.append("Column#")
    Final_List.append(col)
    Name_Space="Anonymous NameSpace"
    File_to_Array(file_name)
    filename = tu.cursor.spelling
    for c in tu.cursor.walk_preorder():
       if c.location.file is None:
            pass
       elif c.location.file.name != filename:
            pass
       elif c.kind == clang.cindex.CursorKind.NAMESPACE:
            Name_Space=c.spelling
       elif c.kind == clang.cindex.CursorKind.FUNCTION_DECL or c.kind==clang.cindex.CursorKind.CXX_METHOD or c.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE:
            Calling_Func= c.spelling
            lin1, col1 = Extract_Line_Column(c)
            st1=File_Content_Array[int(lin1)-1]
            col1=col1[:-1]
       elif (c.kind == clang.cindex.CursorKind.DECL_REF_EXPR or  c.kind == clang.cindex.CursorKind.CALL_EXPR or  c.kind == clang.cindex.CursorKind.OVERLOADED_DECL_REF or  c.kind==clang.cindex.TokenKind.IDENTIFIER) and c.spelling in Function_List:
            Called_Func= c.spelling
            lin2, col2 = Extract_Line_Column(c)
            st2=File_Content_Array[int(lin2)-1]
            col2=col2[:-1]
            col=[]
            col.append(Calling_Func)
            col.append(Name_Space)
            col.append(st1)
            col.append(lin1)
            col.append(col1)
            col.append(Called_Func)
            col.append(st2)
            col.append(lin2)
            col.append(col2)
            if col not in Final_List:
                Final_List.append(col)
            else:
                col=[]
    return Final_List
idx = clang.cindex.Index.create()
os.chdir("/Users/shussain/code-analysis/dynamic/tests")
tu = idx.parse("example2.c", args='-xc++ --std=c++11'.split())
List_of_Function_Calling = Extract_Function_Definition_Calls("example2.c", tu)
for f in List_of_Function_Calling:
    print (f)

### Script-6 (Function_Metrics.py) : Retreive the set of metrics related to functions of example2.c

In [69]:
import csv, os, glob
import sys
import clang.cindex
from clang.cindex import Config
os.environ['DYLD_LIBRARY_PATH']= '/usr/local/Cellar/llvm/11.0.0/lib/'
File_Content_Array=[]
Function_Complexity=[]
List_of_Functions=[]
Function_List=[]
Function_Identifiers=[]
Function_Literals=[]
Cursor_Types = {clang.cindex.CursorKind.IF_STMT, clang.cindex.CursorKind.WHILE_STMT, clang.cindex.CursorKind.FOR_STMT,clang.cindex.CursorKind.DEFAULT_STMT,clang.cindex.CursorKind.CASE_STMT, clang.cindex.CursorKind.COMPOUND_STMT}
Keywords= {"if", "case",  "default", "for", "while", "else"}
def File_to_Array(file_name):
    with open(file_name) as file:
        for line in file:
           File_Content_Array.append(line)
    #return File_Content_Array

def Extract_Identifiers_Literals(tu):
    cnt=0
    filename = tu.cursor.spelling
    for c in tu.cursor.get_tokens():
        #filename = tu.cursor.spelling
        if c.location.file is None:
            pass
        elif c.location.file.name != filename:
            pass
       # elif c.kind==clang.cindex.TokenKind.IDENTIFIER and c.spelling in Function_List:
        elif (c.cursor.kind==clang.cindex.CursorKind.FUNCTION_DECL or c.cursor.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE) and c.spelling in Function_List:
                   # flag=True
                    # fn=c.spelling
                     cnt=cnt+1
        elif c.kind==clang.cindex.TokenKind.IDENTIFIER and c.spelling not in Function_List:
                     temp=Function_Identifiers[cnt-1]
                     temp=temp+1
                     Function_Identifiers[cnt-1]=temp
        elif c.kind==clang.cindex.TokenKind.LITERAL:
                     temp=Function_Literals[cnt-1]
                     temp=temp+1
                     Function_Literals[cnt-1]=temp
def Extract_Function_Complexity(tu):
     cnt=0
     flag=False
     filename = tu.cursor.spelling
     fn=""
     for c in tu.cursor.get_tokens():
         if c.location.file is None:
              pass
         elif c.location.file.name != filename:
              pass
         else:
              cond1=c.cursor.kind in Cursor_Types
              cond2=c.spelling in Keywords
              if (c.cursor.kind==clang.cindex.CursorKind.FUNCTION_DECL or c.cursor.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE) and c.spelling in Function_List:
                     flag=True
                     fn=c.spelling
                     cnt=cnt+1
              if cond1==True and cond2==True and (cnt<=len(Function_Complexity) and Function_List[cnt-1] ==fn):# and len(Function_List)==len(Function_Complexity)):
                     temp=Function_Complexity[cnt-1]
                     temp=temp+1
                     Function_Complexity[cnt-1]=temp
def Extract_Line_Column(cursor):
    x=str(cursor.location)
    y=x.split(',')
    line = y[1]
    column=y[2]
    line = line.split()
    line=line[1]
    column=column.split()
    column = column[1]
    return line, column

def Extract_Function_Qualified_Name(cursor):
     if cursor is None:
        return ''
     elif cursor.kind== clang.cindex.CursorKind.TRANSLATION_UNIT:
        return ''
     else:
         res = Extract_Function_Qualified_Name(cursor.semantic_parent)
         if res != '':
            return res + '::' + cursor.spelling
     return cursor.spelling

def Merge_Function_Complexity(Functions, Complexity, Identifiers, literals):
    Final_List=[]
    # Heading row for the list of functions
    col=[]
    col.append("File Name")
    col.append("Function Name")
    col.append("Function Qualified Name")
    col.append("Mangled Name")
    col.append("Name Space")
    col.append("Function Definition")
    col.append("Line Number")
    col.append("Column Number")
    col.append("Cyclomatic Complexity")
    col.append("Total_Identifiers")
    col.append("Total_Literals")
    Final_List.append(col)
    cnt=0
    for func in Functions:
        col=[]
        col.append(func[0])
        col.append(func[1])
        col.append(func[2])
        col.append(func[3])
        col.append(func[4])
        col.append(func[5])
        col.append(func[6])
        col.append(func[7])
        col.append(Complexity[cnt])
        col.append(Identifiers[cnt])
        col.append(literals[cnt])
        Final_List.append(col)
        cnt=cnt+1
    return Final_List
def Extract_Function_Definition_Location(file_name, tu):
    Name_Space="Anonymous NameSpace"
    File_to_Array(file_name)
    filename = tu.cursor.spelling
    for c in tu.cursor.walk_preorder():
       if c.location.file is None:
            pass
       elif c.location.file.name != filename:
            pass
       elif c.kind == clang.cindex.CursorKind.NAMESPACE:
            Name_Space=c.spelling
       elif c.kind == clang.cindex.CursorKind.FUNCTION_DECL or c.kind==clang.cindex.CursorKind.CXX_METHOD or c.kind==clang.cindex.CursorKind.FUNCTION_TEMPLATE:
            col=[]
            col.append(file_name)
            col.append(c.spelling)
            col.append(Extract_Function_Qualified_Name(c))
            col.append(c.mangled_name)
            col.append(Name_Space)
            lin1, col1 = Extract_Line_Column(c)
            col1=col1[:-1]
            st=File_Content_Array[int(lin1)-1]
            col.append(st.strip())
            col.append(lin1)
            col.append(col1)
            List_of_Functions.append(col)
            Function_List.append(c.spelling)
            Function_Complexity.append(0)
            Function_Identifiers.append(0)
            Function_Literals.append(0)
    Extract_Function_Complexity(tu)
    Extract_Identifiers_Literals(tu)
    Final_List = Merge_Function_Complexity(List_of_Functions, Function_Complexity, Function_Identifiers, Function_Literals)
    return Final_List
idx = clang.cindex.Index.create()
os.chdir("/Users/shussain/code-analysis/dynamic/tests")
tu = idx.parse("example2.c", args='-xc++ --std=c++11'.split())
List_of_Function_Metrics = Extract_Function_Definition_Location("example2.c", tu)
for f in List_of_Function_Metrics:
    print (f)


                    



['File Name', 'Function Name', 'Function Qualified Name', 'Mangled Name', 'Name Space', 'Function Definition', 'Line Number', 'Column Number', 'Cyclomatic Complexity', 'Total_Identifiers', 'Total_Literals']
['example2.c', 'Large', 'Large', '__Z5Largeii', 'Anonymous NameSpace', 'void Large(int a, int b)', '1', '6', 2, 8, 2]
['example2.c', 'main', 'main', '_main', 'Anonymous NameSpace', 'void main() {', '9', '6', 2, 9, 6]
